## Conversion to fif: BJH018

This script takes the raw, original data file we recieved from WashU and converts it to a fif file with the photodiode as a new stim channel


Just need to change fields in `prep_paths`

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [2]:
## prep paths

subject = 'BJH018'
orig_data_fi = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH018/PacmanTask/ECOG001/ECOGS001R02.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}/ieeg"

In [3]:
## load data

with b2k.BCI2kReader(orig_data_fi) as data: #opens a stream to the dat file
    my_signals = data.signals
    my_states = data.states
    my_params = data.parameters

In [4]:
## check structure if necessary

# my_states
my_params

{'SourceCh': 272,
 'SampleBlockSize': 200,
 'SamplingRate': 2000,
 'ChannelNames': ['EMPTY',
  'EMPTY_2',
  'EMPTY_3',
  'EMPTY_4',
  'REF1',
  'REF2',
  'AL1',
  'AL2',
  'AL3',
  'AL4',
  'AL5',
  'AL6',
  'AL7',
  'AL8',
  'AL9',
  'AL10',
  'AL11',
  'AL12',
  'AL13',
  'AL14',
  'AL15',
  'AL16',
  'BL1',
  'BL2',
  'BL3',
  'BL4',
  'BL5',
  'BL6',
  'BL7',
  'BL8',
  'BL9',
  'BL10',
  'BL11',
  'BL12',
  'BL13',
  'BL14',
  'CL1',
  'CL2',
  'CL3',
  'CL4',
  'CL5',
  'CL6',
  'CL7',
  'CL8',
  'CL9',
  'CL10',
  'CL11',
  'CL12',
  'DL1',
  'DL2',
  'DL3',
  'DL4',
  'DL5',
  'DL6',
  'DL7',
  'DL8',
  'DL9',
  'DL10',
  'DL11',
  'DL12',
  'DL13',
  'DL14',
  'EL1',
  'EL2',
  'EL3',
  'EL4',
  'EL5',
  'EL6',
  'EL7',
  'EL8',
  'EL9',
  'EL10',
  'EL11',
  'EL12',
  'EL13',
  'EL14',
  'FL1',
  'FL2',
  'FL3',
  'FL4',
  'FL5',
  'FL6',
  'FL7',
  'FL8',
  'GL1',
  'GL2',
  'GL3',
  'GL4',
  'GL5',
  'GL6',
  'GL7',
  'GL8',
  'HL1',
  'HL2',
  'HL3',
  'HL4',
  'HL5',
  'H

In [5]:
## create info field 

info = mne.create_info(ch_names = my_params['ChannelNames'],
                       sfreq = my_params['SamplingRate'],
                       ch_types = 'seeg')

info

<Info | 7 non-empty values
 bads: []
 ch_names: EMPTY, EMPTY_2, EMPTY_3, EMPTY_4, REF1, REF2, AL1, AL2, AL3, ...
 chs: 272 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 272
 projs: []
 sfreq: 2000.0 Hz
>

In [6]:
## create mne object
mne_df = mne.io.RawArray(my_signals, info)

Creating RawArray with float64 data, n_channels=272, n_times=3121200
    Range : 0 ... 3121199 =      0.000 ...  1560.600 secs
Ready.


In [7]:
## create stim channel and add it to mne object

info = mne.create_info(['STI'], my_params['SamplingRate'], ['stim'])
stim_raw = mne.io.RawArray( my_states['Trial_on_off'], info)
mne_df.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=3121200
    Range : 0 ... 3121199 =      0.000 ...  1560.600 secs
Ready.


<RawArray | 273 x 3121200 (1560.6 s), ~6.35 GB, data loaded>

In [8]:
## save it

mne_df.save(f"{raw_data_dir}/{subject}_raw_ieeg.fif", overwrite = False)

Writing /home/brooke/pacman/raw_data/BJH018/ieeg/BJH018_raw_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH018/ieeg/BJH018_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH018/ieeg/BJH018_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH018/ieeg/BJH018_raw_ieeg.fif
[done]
